In [396]:
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
from urllib.parse import urlencode
import plotly.express as px

# CoinGlass

In [438]:
base_url = "https://open-api.coinglass.com/api/pro/v1/futures/funding_rates_chart?"
params = {
  'symbol' :'ETH',
  'type': 'U' # C for coin, U for USDT
}
headers = {
  'coinglassSecret': '13b32ae9fa584bf39d6ee86880c0c35b'
}

query_string = urlencode(params, True)

url = base_url+query_string
print(url)
response = requests.request("GET", url, headers=headers)
response = response.json()

# dataframe
data = pd.DataFrame()
for ex in response['data']['frDataMap'].keys():
    data[ex] = response['data']['frDataMap'][ex]
data['date'] = response['data']['dateList'][:-1]
data.date = pd.to_datetime(data.date/1000, unit='s')
data.set_index('date', inplace=True)

https://open-api.coinglass.com/api/pro/v1/futures/funding_rates_chart?symbol=ETH&type=U


In [439]:
# data.describe().round(4).to_csv('coinglass.csv')
# data.apply(lambda x: x - data.Binance, axis = 0).describe().round(4).to_csv('coinglass.csv',index = True)

# annual return
# (1+data.apply(lambda x: x - data.Binance, axis = 0).mean()*0.01*3)**365 - 1

# EDA

In [441]:
plot_data = []

for i in range(100):
    a = (data.apply(lambda x: x - data.Binance, axis = 0)).iloc[np.random.randint(0,539,size=50),].mean().tolist()
    plot_data.append(a)
    
plot_data = pd.DataFrame(plot_data,columns = [data.columns])
plot_data = pd.melt(plot_data,var_name= 'exchanges', value_name='value')

fig = px.box(plot_data, x='exchanges', y='value',title=f"{params['symbol']} funding rates between exchanges (X - Binance)")
fig.show()

In [444]:
# fig.write_html('assets/plotly_coinglass_ETHUSDT.html')

In [445]:
(1+0.0162*0.01)**365 - 1

0.0609080697709079

In [8]:
from Exchanges.Okx import Okx
from Exchanges.Binance import Binance
from Exchanges.Gate import Gate
from pandas import json_normalize

In [9]:
okx = Okx()
url_path = '/api/v5/public/instruments'
payload = {
    'instType': 'SWAP',
}
r = okx.send_public_request(url_path, payload)
# print(r)
okx_ex = json_normalize(r['data'])
okx_ex.uly = okx_ex.uly.apply(lambda x: x.replace('-',''))


binance = Binance(BASE_URL= 'https://fapi.binance.com')
url_path = '/fapi/v1/exchangeInfo'
payload = {}
r = binance.send_public_request(url_path, payload=payload)
# print(r)
binance_ex = json_normalize(r['symbols'])


gate = Gate()
url = '/futures/usdt/contracts'
params = {}
bot = Gate()
r = bot.send_public_request(url, params)
# print(r)
gate_ex = json_normalize(r)
gate_ex.name = gate_ex.name.apply(lambda x: x.replace('_',''))

https://www.okx.com/api/v5/public/instruments?instType=SWAP


# Binance and Okx

In [10]:
global exchange_info
exchange_info = (
    pd.merge(okx_ex, binance_ex, left_on='uly', right_on='symbol',how= 'inner')
    .loc[:,['symbol','baseAsset','quoteAsset','uly']]
    .rename(columns={'symbol':'Binance','uly':'Okx'})
)

exchange_info.Okx = exchange_info.Okx.apply(lambda x: x.split('U')[0] + '-'+'U'+ x.split('U')[1])

In [11]:
exchange_info

,Binance,baseAsset,quoteAsset,Okx
0,BTCUSDT,BTC,USDT,BTC-USDT
1,ETHUSDT,ETH,USDT,ETH-USDT
2,LTCUSDT,LTC,USDT,LTC-USDT
3,DOTUSDT,DOT,USDT,DOT-USDT
4,DOGEUSDT,DOGE,USDT,DOGE-USDT
...,...,...,...,...
78,YFIUSDT,YFI,USDT,YFI-USDT
79,ZECUSDT,ZEC,USDT,ZEC-USDT
80,ZENUSDT,ZEN,USDT,ZEN-USDT
81,ZILUSDT,ZIL,USDT,ZIL-USDT


# Gate and Binance

In [12]:
exchange_info = (
    pd.merge(gate_ex, binance_ex, left_on='name', right_on='symbol',how= 'inner')
    .loc[:,['symbol','baseAsset','quoteAsset','name']]
    .rename(columns={'symbol':'Binance','name':'gate'})
)
exchange_info.gate = exchange_info.gate.str[:-4] + "_"+ exchange_info.gate.str[-4:] 

In [13]:
exchange_info

,Binance,baseAsset,quoteAsset,gate
0,KNCUSDT,KNC,USDT,KNC_USDT
1,SCUSDT,SC,USDT,SC_USDT
2,ZECUSDT,ZEC,USDT,ZEC_USDT
3,ICXUSDT,ICX,USDT,ICX_USDT
4,RVNUSDT,RVN,USDT,RVN_USDT
...,...,...,...,...
131,ZENUSDT,ZEN,USDT,ZEN_USDT
132,IOSTUSDT,IOST,USDT,IOST_USDT
133,ARPAUSDT,ARPA,USDT,ARPA_USDT
134,EGLDUSDT,EGLD,USDT,EGLD_USDT


In [272]:
def gate_get_FundingRateHist(symbol):
    gate = Gate()
    url = '/futures/usdt/funding_rate'
    
    params = {
        'contract':symbol,
        'limit':1000
    }
    
    bot = Gate()
    r = bot.send_public_request(url, params)
    
    gate_fundingrate = (
        json_normalize(r)
        .rename(columns={'t':'time','r':'gate'})
        .assign(time = lambda x: pd.to_datetime(x.time, unit='s'))
        .assign(gate = lambda x: x.gate.astype(float))
    )
    return gate_fundingrate
    
def binance_get_FundingRateHist(symbol, endtime = None):

    # endtime = '2021-09-16 00:10:00'
    binance = Binance(BASE_URL= 'https://fapi.binance.com')
    url_path = '/fapi/v1/fundingRate'
    
    if not endtime:
        params = {
            'symbol':symbol,
            'limit':1000
        }
    else:
        starttime = int((pd.to_datetime(endtime) - pd.Timedelta(hours=8)*1000).timestamp()*1000)
        # endtime = int(datetime.datetime.strptime(endtime, "%Y-%m-%d %H:%M:%S").timestamp()*1000)
        params = {
            'symbol':symbol,
            'limit':1000,
            'startTime':starttime,
        }
        # print(params) 
        
    r = binance.send_public_request(url_path, payload=params)
    # return r
    
    binance_fundingrate = (
        json_normalize(r)
        .iloc[:,1:]
        .rename(columns={'fundingTime':'time','fundingRate':'binance'})
        .assign(time = lambda x: pd.to_datetime(x.time/1000, unit='s')) 
        .assign(time = lambda x: pd.to_datetime(x.time.dt.strftime('%Y-%m-%d %H:%M:%S')))
        .assign(binance = lambda x: x.binance.astype(float))
    )
    return binance_fundingrate


def get_evaluation_symbol(all_data, asset, test_point = None, train_span = None):
    
    # test_point = -30 # is the current starting point for the test set, if -1, it will be the newest day, if -2, will be the last day, etc.
    # train_span = 1 # is the time span for the training set, if 1, the training data set will be the last day, if 2, will be last two days, etc.
    
    if  test_point and train_span:
    
    #* other measurements
        # .assign(pos_count = lambda x: eval(((x.gate - x.binance)>0).value_counts().to_json())['True'])
        # .assign(neg_count = lambda x: eval(((x.gate - x.binance)>0).value_counts().to_json())['False'])
        # .assign(pos_neg_ratio = lambda x: eval(((x.gate - x.binance)>0).value_counts().to_json())['True']/eval(((x.gate - x.binance)<0).value_counts().to_json())['False'])

        if (-test_point) > all_data[asset].shape[0]:
            return None
        
        symbol_evaluation = (
            all_data[asset]
            .sort_values('time', ascending = True).reset_index(drop = True)
            .iloc[test_point - train_span:test_point,:]
            .assign(mean_difference = lambda x: (x.gate - x.binance).mean())
            .assign(annual_ret = lambda x: ((x.gate - x.binance).mean()*3 + 1)**365 -1)
            .assign(asset = asset)
            .iloc[-1,:]
            .to_frame()
            .T
        )
    else:
        symbol_evaluation = (
            all_data[asset]
            .sort_values('time', ascending = True).reset_index(drop = True)
            .assign(mean_difference = lambda x: (x.gate - x.binance).mean())
            .assign(annual_ret = lambda x: ((x.gate - x.binance).mean()*3 + 1)**365 -1)
            .assign(asset = asset)
            .iloc[-1,:]
            .to_frame()
            .T
        )
    return symbol_evaluation


def get_all_fundingrate():
    all_data = {}
    
    for asset in exchange_info.baseAsset:
        
        # asset = 'ZEC'
        try: 
            binance_symbol = exchange_info.loc[exchange_info.baseAsset == asset, 'Binance'].values[0]
            gate_symbol = exchange_info.loc[exchange_info.baseAsset == asset, 'gate'].values[0]
            
            gate_fundingrate = gate_get_FundingRateHist(gate_symbol)
            binance_fundingrate  = binance_get_FundingRateHist(binance_symbol)
            
            all_data[asset] = pd.merge(gate_fundingrate, binance_fundingrate, on ='time' ,how= 'inner').sort_values('time', ascending = True).reset_index(drop = True)
        except Exception as e:
            continue
    
    return all_data
    

def get_evaluation_table(all_data, test_point, train_span):

    all_evaluation = pd.DataFrame()

    for asset in exchange_info.baseAsset:
        # print(asset)
        symbol_evaluation = get_evaluation_symbol(all_data, asset, test_point, train_span)
        all_evaluation = all_evaluation.append(symbol_evaluation)
        
    return all_evaluation.reset_index(drop=True)

def get_target_pair(all_data, test_point, train_span):
    all_evaluation = get_evaluation_table(all_data, test_point, train_span)
    pos_pair = all_evaluation.sort_values('annual_ret', ascending = False).reset_index(drop = True)['asset'].to_numpy()[[0,-1]][0]
    neg_pair = all_evaluation.sort_values('annual_ret', ascending = False).reset_index(drop = True)['asset'].to_numpy()[[0,-1]][1]
    
    """
    
    pos_pair -> high positive in exchange 1 -> earn funding rate in exchange -> take short because of positive funding rate -> hedge with long in binance
    
    pos_pair: the funding rate in exchange 1 is larger than that in binance, where we should take short of pos_pair in exchange 1, and short of pos_pair in binance.
    neg_pair: the funding rate in exchange 1 is small than that in binance, where we should take long in exchange 1 and long in 
    """
    
    return [pos_pair, neg_pair]
    
    
def obtain_symbol_return(all_data, asset, test_point):

    ret = (
        all_data[asset]
        .sort_values('time', ascending = True).reset_index(drop = True)
        .iloc[test_point:test_point+1,:]
        .assign(difference = lambda x: x.gate - x.binance)
        .assign(asset = asset)
    )
    
    return ret
# all_data = get_all_fundingrate()

In [393]:
validation_data = []
for train_span in range(1,100):
    test_point = -300
    # train_span = 20
    ret_table = pd.DataFrame()
    while test_point <0:
        # select trading pair
        pos_pair, neg_pair = get_target_pair(all_data, test_point, train_span)
        # print(pos_pair)
        
        mid_tm = test_point + 20
    
        while test_point < mid_tm:
            # calculate the daily return of the trading pair
            day_ret = obtain_symbol_return(all_data, pos_pair, test_point)
            ret_table = ret_table.append(day_ret)
            test_point += 1
            if test_point >= 0:
                break
            # print(test_point)
            
    ret_table = ret_table.reset_index(drop = True)
    test_ret = (1 + ret_table.difference.mean() * 3) ** 365 - 1
    validation_data.append([train_span,test_ret])
    

In [394]:
plot_data = pd.DataFrame(validation_data,columns = ['train_span','test_ret'])

In [395]:
import plotly.express as px

fig = px.line(plot_data, x='train_span', y='test_ret', title="Validation for training span")
fig.show()

# Plotting module for single asset

In [530]:
asset = 'BAL'

binance_symbol = exchange_info.loc[exchange_info.baseAsset == asset, 'Binance'].values[0]
gate_symbol = exchange_info.loc[exchange_info.baseAsset == asset, 'gate'].values[0]

gate_fundingrate = gate_get_FundingRateHist(gate_symbol)
binance_fundingrate  = binance_get_FundingRateHist(binance_symbol)

In [531]:
plot_data = (
    pd.merge(gate_fundingrate, binance_fundingrate, on ='time' ,how= 'inner')
    .melt(id_vars=['time'], value_vars=['gate','binance'], var_name='exchanges', value_name='value')
    # .set_index('time',drop = True)
    .assign(value = lambda x: x.value*100)
)

import plotly.express as px

fig = px.bar(plot_data, x='time', y='value', color="exchanges", title=f"Funding rates between exchanges (%), symbol: {asset}", barmode = 'group')
fig.show()

# Backtesting module

## step1: obtain the funding rate till today

In [209]:
get_target_pair(all_data, test_point, train_span)

['RSR', 'STORJ']

In [212]:
get_evaluation_table(all_data, test_point, train_span).sort_values(ascending = False, by = 'annual_ret')

,time,gate,binance,mean_difference,annual_ret,asset
38,2022-07-06,-0.000496,-0.000276,0.000463,0.660202,RSR
97,2022-07-06,0.0001,0.00005,0.000401,0.551441,BAL
23,2022-07-06,-0.003287,-0.004429,0.000335,0.442903,OGN
9,2022-07-06,0.0001,0.0001,0.000311,0.405262,KLAY
102,2022-07-06,0.0001,-0.000007,0.000282,0.36094,KSM
...,...,...,...,...,...,...
17,2022-07-06,0.0001,0.0001,-0.000188,-0.186109,DENT
123,2022-07-06,0.000077,0.0001,-0.000194,-0.191585,RUNE
30,2022-07-06,-0.000302,-0.000461,-0.000211,-0.206281,MTL
98,2022-07-06,-0.000571,0.0001,-0.000246,-0.235872,SXP
